In [1]:
from utils import get_files, write_json
from datetime import date
from collections import defaultdict

CACHE = get_files()
authors_registry, authors_registry_path = CACHE["authors_registry"]
outlines_jsonl, outlines_jsonl_path = CACHE["outlines_jsonl"]
theologians, theologians_path = CACHE["theologians"]
topic_mapping, topic_mapping_path = CACHE["topic_mapping"]
topics, topics_path = CACHE["topics"]
traditions, traditions_path = CACHE["traditions"]
eras, eras_path = CACHE["eras"]
work_canon_map, work_canon_map_path = CACHE["work_canon_map"]
works, works_path = CACHE["works"]
by_theologian, by_theologian_path = CACHE["by_theologian"]
by_topic, by_topic_path = CACHE["by_topic"]
by_topic_keyworks, by_topic_keyworks_path = CACHE["by_topic_keyworks"]
by_work, by_work_path = CACHE["by_work"]
eras_registry, eras_registry_path = CACHE["eras_registry"]
institutions_registry, institutions_registry_path = CACHE["institutions_registry"]
theologian_profiles, theologian_profiles_path = CACHE["theologian_profiles"]
topic_work_edges, topic_work_edges_path = CACHE["topic_work_edges"]
search_index, search_index_path = CACHE["search_index"]

theo_ids = [t["id"] for t in theologians]
topic_data = {t["id"]: t for t in topics}
theo_outs = {out["theologian_id"]: [o for o in outlines_jsonl if o["theologian_id"] == out["theologian_id"]] for out in
             outlines_jsonl}
extra_works = {tid: [w for w, wdata in by_work.items() if not wdata["referenced_in"] and wdata["primary_author_theologian_id"] == tid] for tid in theo_ids}
theo_all_works = {tid: [w for w, wdata in by_work.items() if wdata["primary_author_theologian_id"] == tid] for tid in theo_ids}

new_by_theologian = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
# new_by_theologian = {t: {} for t in theo_ids}

new_theologians = []


for theo in theologians:
    tid = theo["id"]
    theo_outlines = theo_outs.get(tid, [])


    for theo_outline in theo_outlines:
        outline_id = theo_outline["id"]

        topic_id = theo_outline["topic_id"]
        topic_slug = topic_data[topic_id]["slug"]
        topic_title = topic_data[topic_id]["title"]
        topic_category = topic_data[topic_id]["category"]
        key_work_ids = theo_outline["key_work_ids"]
        mark_down_path = theo_outline["markdown_path"]
        updated_at = theo_outline["updated_at"]

        new_by_theologian[tid]["outlines_by_topic_category"][topic_category].append(
            {
                "topic_id": topic_id,
                "topic_slug": topic_slug,
                "topic_category": topic_category,
                "markdown_path": mark_down_path,
                "updated_at": updated_at,
                "key_work_ids": key_work_ids,
                "outline_id": outline_id,
            }
        )


for theo in theologians:
    tid = theo["id"]
    new_by_theologian[tid]["key_work_ids"] = extra_works.get(tid, [])

    theo["key_work_ids"] = theo_all_works[tid]
    new_theologians.append(theo)




write_json("data/indices/by_theologian-test.json", new_by_theologian)
write_json("data/theologian-test.json", new_theologians)

In [ ]:
new_by_theologian

In [ ]:
topic_data = {t["id"]: t for t in topics}
topic_data["top_73dd29f4e3dd"]


In [ ]:
from collections import defaultdict

new_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

new_dict["a"]["b"]["c"].append("3")

In [ ]:
works[0]

In [ ]:
a = list(by_work.keys())[0]
by_work[a]

In [ ]:
for w, wdata in by_work.items():
    if not wdata["referenced_in"]:
        print(wdata)
        break

In [ ]:
extra_works = {tid: [w for w, wdata in by_work.items() if not wdata["referenced_in"] and wdata["primary_author_theologian_id"] == tid] for tid in [t["id"] for t in theologians]}

In [ ]:
extra_works

In [ ]:
new_by_theologian